<a href="https://colab.research.google.com/github/DaeSeokSong/MachineLearningModels/blob/main/BlackUP_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Drive Local Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

# Main Code

In [ ]:
%cd drive/MyDrive/DeepLearning/BlackUp/Dataset

studyData = pd.read_csv("KOSPI200_20y_dataset.csv");
testData = pd.read_csv("005930_10y_dataset.csv")

chageRate = studyData["ChageRate"].values

# Window size
seq_len = 50
total_seq_len = seq_len + 1 # 예측 기반 데이터 (seq_len)개 + 예측할 데이터 1개

result = []
for idx in range(len(chageRate) - total_seq_len) :
    result.append(chageRate[idx: idx + total_seq_len])

result = np.array(result)

# 테스트셋 / 훈련셋 분류
row = int(round(result.shape[0] * 0.9))
# 훈련셋 분류 및 랜덤 셔플
train = result[:row, :]
np.random.shuffle(train)

# 훈련셋
x_train = train[:, :-1]
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
y_train = train[:, -1]

# 테스트셋
x_test = result[row:, :-1]
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
y_test = result[row:, -1]

# keras.models 중 모델을 순차적으로 정의하는 클래스
model = Sequential()

model.add(LSTM(
    1, # 해당 층의 노드 개수
    input_shape=(50,1), # input_shape=입력값 모양
    return_sequences=True)) # return_sequences == 각 시퀀스를 출력할지
model.add(Dropout(0.2)) # 과적합 방지용 Dropout 20%(==0.2)

model.add(LSTM(
    100, # 노드 100개
    return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(1, activation='sigmoid')) # 활성화(Activation)함수 = sigmoid
model.compile(loss='mse', optimizer='adam') # 최적화함수 = adam

model.fit(x_train, y_train, # 훈련셋
          validation_data=(x_test, y_test), # 테스트셋, validation_data
          batch_size=128, # 한 번에 몇 개씩 묶어서 학습시킬지, batch_size
          epochs=4) # N번 동안 반복해라, epochs

# 예측
pred = model.predict(x_test)

plt.plot(x_test, color="r", label="x_test")
#plt.plot(y_test, color="b", label="y_test")
plt.title("Predict ChangeRate Test")
plt.legend()
plt.show()